### new partition

In [16]:
from utilities import *

all_files = glob.glob(os.path.join(TRAIN_AUDIO_DIR, '*/*wav'))

with open(os.path.join(TRAIN_DIR, 'validation_list.txt'), 'r') as fin:
    validation_files = fin.readlines()
valset = set()
for fname in validation_files:
    splits = fname.split('/')
    valset.add(splits[-1].split('_')[0].rstrip())
train, val = [], []
noise = [(NAME2ID['silence'], '', '')]
for fname in all_files:
    splits = fname.split('\\')
    label, uid = splits[-2], splits[-1].split('_')[0]
    if label == '_background_noise_':
        label = 'silence'

    sample = (label, uid, fname)
    if uid in valset:
        val.append(sample)
    elif label == 'silence':
        noise.append(sample)
    else:
        train.append(sample)

print('There are {} train and {} val samples'.format(len(train), len(val)))


There are 57923 train and 6798 val samples


In [23]:
from collections import Counter
count_train = Counter([s[0] for s in train])
for k in count_train.keys():
    print('{}:\t{},\t{}'.format(k, count_train[k], count_train[k] / len(train)))

four:	2092,	0.036116913833882915
stop:	2134,	0.03684201439842549
yes:	2116,	0.03653125701362153
dog:	1576,	0.027208535469502616
right:	2111,	0.036444935517842655
left:	2106,	0.03635861402206377
bed:	1516,	0.02617267752015607
five:	2115,	0.03651399271446576
three:	2108,	0.036393142620375324
bird:	1569,	0.027087685375412184
wow:	1579,	0.027260328366969944
on:	2110,	0.036427671218686876
happy:	1553,	0.02681145658891977
no:	2105,	0.036341349722908
six:	2107,	0.03637587832121955
nine:	2134,	0.03684201439842549
up:	2115,	0.03651399271446576
two:	2137,	0.036893807295892825
seven:	2114,	0.03649672841530998
go:	2112,	0.03646219981699843
cat:	1565,	0.027018628178789084
one:	2140,	0.03694560019336015
house:	1577,	0.027225799768658392
down:	2095,	0.03616870673135024
zero:	2116,	0.03653125701362153
tree:	1567,	0.027053156777100632
marvin:	1586,	0.027381178461060373
off:	2101,	0.036272292526284895
sheila:	1558,	0.02689777808469865
eight:	2109,	0.0364104069195311


In [24]:
count_val = Counter([s[0] for s in val])
for k in count_val.keys():
    print('{}:\t{},\t{}'.format(k, count_val[k], count_val[k] / len(val)))

four:	280,	0.04118858487790527
stop:	246,	0.03618711385701677
yes:	261,	0.038393645189761696
dog:	170,	0.025007355104442484
right:	256,	0.03765813474551338
left:	247,	0.03633421594586643
bed:	197,	0.028979111503383347
five:	242,	0.03559870550161812
three:	248,	0.03648131803471609
bird:	162,	0.02383053839364519
wow:	166,	0.024418946749043838
on:	257,	0.03780523683436305
happy:	189,	0.027802294792586054
no:	270,	0.03971756398940865
six:	262,	0.038540747278611356
nine:	230,	0.033833480435422184
up:	260,	0.03824654310091203
two:	236,	0.03471609296852015
seven:	263,	0.038687849367461016
go:	260,	0.03824654310091203
cat:	168,	0.02471315092674316
one:	230,	0.033833480435422184
house:	173,	0.025448661370991468
down:	264,	0.038834951456310676
zero:	260,	0.03824654310091203
tree:	166,	0.024418946749043838
marvin:	160,	0.023536334215945868
off:	256,	0.03765813474551338
sheila:	176,	0.025889967637540454
eight:	243,	0.03574580759046778


In [78]:
all_files = glob.glob(os.path.join(TRAIN_MODIFIED_AUDIO_DIR, '*/*wav'))
uids = []
f_sil = []
for fname in all_files:
    splits = fname.split('\\')
    label, uid = splits[-2], splits[-1].split('_')[0]
    if label == '_background_noise_':
        f_sil.append(fname)
    else:
        uids.append(uid)
uids = np.unique(uids)
f_sil = np.unique(f_sil)

In [80]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=17)
uids_folds = []
for train_index, test_index in kf.split(uids):
    uids_folds.append([uids[train_index], uids[test_index]])

sil_folds = []
for train_index, test_index in kf.split(f_sil):
    sil_folds.append([f_sil[train_index], f_sil[test_index]])

In [82]:
for uid_f in uids_folds:
    print (len(uid_f[0]), len(uid_f[1]))
    for i in uid_f[0]:
        assert i not in uid_f[1]
        
for f in sil_folds:
    print (len(f[0]), len(f[1]))
    for i in f[0]:
        assert i not in f[1]

1409 470
1409 470
1409 470
1410 469
6264 2089
6265 2088
6265 2088
6265 2088


In [83]:
all_files = glob.glob(os.path.join(TRAIN_MODIFIED_AUDIO_DIR, '*/*wav'))
folds = []
for i in range(len(uids_folds)):
    uid_f = uids_folds[i]
    sil_f = sil_folds[i]
    
    train_f = []
    test_f = []
    for fname in all_files:
        splits = fname.split('\\')
        label, uid = splits[-2], splits[-1].split('_')[0]
        if uid in uid_f[0] or fname in sil_f[0]:
            train_f.append(fname)
        elif uid in uid_f[1] or fname in sil_f[1]:
            test_f.append(fname)
        else:
            print(uid)
            print('WTF?!')
    folds.append([train_f, test_f])

In [92]:
for f in folds:
    print (len(f[0]), len(f[1]))
    for i in f[0]:
        assert i not in f[1]

54171 18056
54556 17671
53922 18305
54599 17628


In [63]:
from collections import Counter

def test_folds(folds):
    for fold in folds:
        train_files = [f.split('\\')[-2] + f.split('\\')[-1] for f in fold[0]]
        val_files = [f.split('\\')[-2] + f.split('\\')[-1] for f in fold[1]]

        all_files = glob.glob(os.path.join(TRAIN_MODIFIED_AUDIO_DIR, '*/*wav'))

        train, val = [], []
        for fname in all_files:
            splits = fname.split('\\')
            label, uid = splits[-2], splits[-1].split('_')[0]
            if label == '_background_noise_':
                label = 'silence'
            sample = (label, uid, fname)
            sample_id = splits[-2] + splits[-1]
            if sample_id in val_files:
                val.append(sample)
            elif sample_id in train_files:
                train.append(sample)
        print('There are {} train and {} val samples'.format(len(train), len(val)))

        count_train = Counter([s[0] for s in train])
        count_val = Counter([s[0] for s in val])
        for k in count_train.keys():
            print('{}:\t{}\t-\t{},\t{}\t-\t{}'.format(k, count_train[k], count_val[k], count_train[k] / len(train), count_val[k] / len(val)))
        print('==============================================================')

In [85]:
test_folds(folds)

There are 54171 train and 18056 val samples
four:	1769	-	573,	0.0326558490705359	-	0.03173460345591493
stop:	1748	-	599,	0.03226818777574717	-	0.033174568010633586
yes:	1776	-	574,	0.032785069502132135	-	0.031789986708019496
dog:	1288	-	453,	0.02377655941370844	-	0.0250886132033673
right:	1729	-	591,	0.031917446604271656	-	0.032731501993797076
left:	1733	-	582,	0.03199128685089808	-	0.03223305272485601
bed:	1280	-	431,	0.023628878920455593	-	0.0238701816570669
five:	1727	-	601,	0.031880526480958446	-	0.03328533451484271
three:	1707	-	558,	0.03151132524782633	-	0.030903854674346476
bird:	1276	-	446,	0.02355503867382917	-	0.024700930438635357
wow:	1269	-	465,	0.02342581824223293	-	0.025753212228622063
on:	1769	-	555,	0.0326558490705359	-	0.030737704918032786
happy:	1275	-	454,	0.023536578612172565	-	0.025143996455471867
no:	1780	-	567,	0.03285890974875856	-	0.03140230394328755
silence:	6367	-	1986,	0.11753521256760997	-	0.10999113867966327
six:	1760	-	564,	0.03248970851562644	-	0.0312361

In [86]:
with gzip.open(TRAIN_MODIFIED_DIR + 'kfold4_max.pklz', 'wb') as f:
    pickle.dump(folds, f)

In [88]:
my_folds = pickle.load(gzip.open(TRAIN_MODIFIED_DIR + 'kfold4_max.pklz', 'rb'))
assert (my_folds == folds)

In [65]:
import pickle
import gzip

our_folds = pickle.load(gzip.open(TRAIN_MODIFIED_DIR + KFOLD_FILENAME, 'rb'))

In [66]:
test_folds(our_folds)

There are 53504 train and 18717 val samples
four:	2342	-	0,	0.043772428229665074	-	0.0
stop:	1768	-	579,	0.033044258373205744	-	0.030934444622535664
dog:	1741	-	0,	0.03253962320574163	-	0.0
right:	1742	-	578,	0.03255831339712919	-	0.030881017257039057
left:	1723	-	592,	0.03220319976076555	-	0.03162900037399156
bed:	1711	-	0,	0.03197891746411483	-	0.0
five:	2328	-	0,	0.043510765550239236	-	0.0
three:	2265	-	0,	0.04233328349282297	-	0.0
bird:	1722	-	0,	0.03218450956937799	-	0.0
yes:	1734	-	616,	0.032408791866028706	-	0.03291125714591014
on:	1724	-	600,	0.03222188995215311	-	0.03205641929796442
happy:	1729	-	0,	0.03231534090909091	-	0.0
no:	1762	-	585,	0.03293211722488038	-	0.03125500881551531
silence:	6040	-	2307,	0.11288875598086125	-	0.12325693220067319
six:	2324	-	0,	0.043436004784688995	-	0.0
seven:	2336	-	0,	0.04366028708133971	-	0.0
up:	1734	-	589,	0.032408791866028706	-	0.03146871827750174
two:	2339	-	0,	0.04371635765550239	-	0.0
sheila:	1720	-	0,	0.03214712918660287	-	0.0
go:	173

In [1]:
from utilities import *
import pickle
import gzip

folds = pickle.load(gzip.open(TRAIN_MODIFIED_DIR + MY_KFOLD_FILENAME, 'rb'))

for i in range(len(folds)):
    trainset, valset = load_fold(folds[i], no_unk=False)
    val_files = [v[2] for v in valset]
    train_files = [v[2] for v in trainset]
    for f in val_files:
        if f in train_files:
            print(f)
        

C:\Users\MAX\Anaconda2\envs\tf\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


There are 54171 train and 18056 val samples
There are 54556 train and 17671 val samples
There are 53922 train and 18305 val samples
There are 54599 train and 17628 val samples


In [2]:
for i in range(len(folds)):
    trainset, valset = load_fold(folds[i], no_unk=True)
    val_files = [v[2] for v in valset]
    train_files = [v[2] for v in trainset]
    for f in val_files:
        if f in train_files:
            print(f)

There are 54171 train and 7767 val samples
There are 54556 train and 7631 val samples
There are 53922 train and 8080 val samples
There are 54599 train and 7617 val samples


In [ ]:
for i in range(len(folds)):
    trainset, valset = load_fold(folds[i], no_unk=False)
    val_uids = np.unique([v[1] for v in valset])
    train_uids = np.unique([v[1] for v in trainset])
    for i in val_uids:
        if i in train_uids:
            print(i)

In [12]:
for i in range(len(folds)):
    trainset, valset = load_fold(folds[i], no_unk=True)
    val_uids = np.unique([v[1] for v in valset])
    train_uids = np.unique([v[1] for v in trainset])
    for i in val_uids:
        if i in train_uids:
            print(i)

There are 54171 train and 7767 val samples
05b2db80
122c5aa7
190821dc
2bdbe5f7
3bfd30e6
3d53244b
533c6564
611d2b50
6727b579
6c968bd9
763188c4
88120683
94de6a6a
953fe1ad
96ab6565
9aa21fa9
alexbluenoise
alexbrownnoise
alexdoingthedishes
alexdudemiaowing
alexexercisebike
alexpinknoise
alexrunningtap
alexvioletnoise
alexwhitenoise
b0bad4a9
b9515bf3
b959cd0c
b9f46737
bab36420
c93d5e22
cd85758f
ce7a8e92
d90b4138
dbb40d24
dea820ce
e7ea8b76
e96a5020
eb3f7d82
ee4a907f
f953e1af
fd395b74
twlgdeye
zfturbo
There are 54556 train and 7631 val samples
0132a06d
099d52ad
2da58b32
35d1b6ee
38d78313
3f2b358d
40115b19
43691f67
53eb0a88
712e4d58
7211390b
7846fd85
7f74626f
7fd25f7c
888a0c49
9ff2d2f4
a6d586b7
alexbluenoise
alexbrownnoise
alexdoingthedishes
alexdudemiaowing
alexexercisebike
alexpinknoise
alexrunningtap
alexvioletnoise
alexwhitenoise
b4aa9fef
d9462202
e0315cf6
e96a5020
ec201020
twlgdeye
zfturbo
There are 53922 train and 8080 val samples
0137b3f4
179a61b7
1bc45db9
1cb788bc
1eddce1d
28ce0c58
28ed

In [15]:
trainset, valset = load_fold(folds[0], no_unk=True)
for s in trainset:
    if s[1] == '122c5aa7':
        print(s)
print('---------------------------------')
for s in valset:
    if s[1] == '122c5aa7':
        print(s)

There are 54171 train and 7767 val samples
(10, '122c5aa7', 'C:/data/tf_speech/train_modified/audio\\_background_noise_\\122c5aa7_nohash_1.wav')
(10, '122c5aa7', 'C:/data/tf_speech/train_modified/audio\\_background_noise_\\122c5aa7_nohash_3.wav')
(10, '122c5aa7', 'C:/data/tf_speech/train_modified/audio\\_background_noise_\\122c5aa7_nohash_4641.wav')
---------------------------------
(3, '122c5aa7', 'C:/data/tf_speech/train_modified/audio\\down\\122c5aa7_nohash_0.wav')
(3, '122c5aa7', 'C:/data/tf_speech/train_modified/audio\\down\\122c5aa7_nohash_1.wav')
(3, '122c5aa7', 'C:/data/tf_speech/train_modified/audio\\down\\122c5aa7_nohash_2.wav')
(3, '122c5aa7', 'C:/data/tf_speech/train_modified/audio\\down\\122c5aa7_nohash_3.wav')
(3, '122c5aa7', 'C:/data/tf_speech/train_modified/audio\\down\\122c5aa7_nohash_4.wav')
(9, '122c5aa7', 'C:/data/tf_speech/train_modified/audio\\go\\122c5aa7_nohash_0.wav')
(9, '122c5aa7', 'C:/data/tf_speech/train_modified/audio\\go\\122c5aa7_nohash_1.wav')
(9, '122c

In [9]:
train_uids

array(['00176480', '00b01445', '00f0204f', ..., 'twlgdeye', 'white',
       'zfturbo'], dtype='<U18')

## KFold: my noise, by speakers

In [61]:
all_files = glob.glob(os.path.join(TRAIN_AUDIO_DIR, '*/*wav'))
uids = []
f_sil = []
for fname in all_files:
    splits = fname.split('\\')
    label, uid = splits[-2], splits[-1].split('_')[0]
    if label == '_background_noise_':
        f_sil.append(fname)
        continue
    else:
        uids.append(uid)
uids = np.unique(uids)
f_sil = np.unique(f_sil)

In [62]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=17)
uids_folds = []
for train_index, test_index in kf.split(uids):
    uids_folds.append([uids[train_index], uids[test_index]])

In [63]:
for uid_f in uids_folds:
    print (len(uid_f[0]), len(uid_f[1]))
    for i in uid_f[0]:
        assert i not in uid_f[1]

1410 471
1411 470
1411 470
1411 470


In [65]:
all_files = glob.glob(os.path.join(TRAIN_AUDIO_DIR, '*/*wav'))
folds = []
for i in range(len(uids_folds)):
    uid_f = uids_folds[i]
    
    train_f = []
    test_f = []
    for fname in all_files:
        splits = fname.split('\\')
        label, uid = splits[-2], splits[-1].split('_')[0]
        if label == '_background_noise_':
            continue
        if uid in uid_f[0]:
            train_f.append(fname)
        elif uid in uid_f[1]:
            test_f.append(fname)
        else:
            print(uid)
            print('WTF?!')
    folds.append([train_f, test_f])

In [92]:
for f in folds:
    print (len(f[0]), len(f[1]))
    for i in f[0]:
        assert i not in f[1]

54171 18056
54556 17671
53922 18305
54599 17628


In [69]:
from collections import Counter

def test_folds(folds):
    for fold in folds:
        train_files = [f.split('\\')[-2] + f.split('\\')[-1] for f in fold[0]]
        val_files = [f.split('\\')[-2] + f.split('\\')[-1] for f in fold[1]]

        all_files = glob.glob(os.path.join(TRAIN_AUDIO_DIR, '*/*wav'))

        train, val = [], []
        for fname in all_files:
            splits = fname.split('\\')
            label, uid = splits[-2], splits[-1].split('_')[0]
            if label == '_background_noise_':
                print(fname)
                continue
            sample = (label, uid, fname)
            sample_id = splits[-2] + splits[-1]
            if sample_id in val_files:
                val.append(sample)
            elif sample_id in train_files:
                train.append(sample)
        print('There are {} train and {} val samples'.format(len(train), len(val)))

        count_train = Counter([s[0] for s in train])
        count_val = Counter([s[0] for s in val])
        for k in count_train.keys():
            print('{}:\t{}\t-\t{},\t{}\t-\t{}'.format(k, count_train[k], count_val[k], count_train[k] / len(train), count_val[k] / len(val)))
        print('==============================================================')

In [103]:
folds = pickle.load(gzip.open(TRAIN_MODIFIED_DIR + MY_KFOLD_FILENAME, 'rb'))

for f in folds:
    print(len(f[0]), len(f[1]))
    
    count = 0
    for fname in f[0]:
        splits = fname.split('\\')
        label, uid = splits[-2], splits[-1].split('_')[0]
        if label == '_background_noise_':
            continue
        count += 1
    print(count)
    
    count = 0
    for fname in f[1]:
        splits = fname.split('\\')
        label, uid = splits[-2], splits[-1].split('_')[0]
        if label == '_background_noise_':
            continue
        count += 1
    print(count)

54171 18056
47804
16070
54556 17671
48125
15749
53922 18305
47525
16349
54599 17628
48168
15706


In [104]:
folds = pickle.load(gzip.open(TRAIN_MODIFIED_DIR + KFOLD_FILENAME, 'rb'))

for f in folds:
    print(len(f[0]), len(f[1]))
    
    count = 0
    for fname in f[0]:
        splits = fname.split('\\')
        label, uid = splits[-2], splits[-1].split('_')[0]
        if label == '_background_noise_':
            continue
        count += 1
    print(count)
    
    count = 0
    for fname in f[1]:
        splits = fname.split('\\')
        label, uid = splits[-2], splits[-1].split('_')[0]
        if label == '_background_noise_':
            continue
        count += 1
    print(count)

53504 18717
47464
16410
53793 18428
47448
16426
54180 18041
47806
16068
55186 17035
48904
14970


In [4]:
from utilities import *
import pickle
import gzip

folds = pickle.load(gzip.open(TRAIN_MODIFIED_DIR + KFOLD_FILENAME, 'rb'))
trainset, valset = load_fold_my_noise(folds[0])

There are 47464 train and 16410 val samples
Noise files:
doing_the_dishes.wav
dude_miaowing.wav
exercise_bike.wav
pink_noise.wav
running_tap.wav
white_noise.wav
There are 51566 train and 17824 val samples after adding SILENCE


In [5]:
from utilities import *
import pickle
import gzip

folds = pickle.load(gzip.open(TRAIN_MODIFIED_DIR + MY_KFOLD_FILENAME, 'rb'))
trainset_my, valset_my = load_fold_my_noise(folds[0])

There are 47804 train and 16070 val samples
Noise files:
doing_the_dishes.wav
dude_miaowing.wav
exercise_bike.wav
pink_noise.wav
running_tap.wav
white_noise.wav
There are 51934 train and 17456 val samples after adding SILENCE


In [13]:
fnames = set()
for s in valset:
    if s[0] == NAME2ID['silence']:
        fnames.add(s[2])
print(fnames)

{'', 'C:/data/tf_speech/train/audio/_background_noise_/white_noise.wav', 'C:/data/tf_speech/train/audio/_background_noise_/running_tap.wav', 'C:/data/tf_speech/train/audio/_background_noise_/dude_miaowing.wav', 'C:/data/tf_speech/train/audio/_background_noise_/doing_the_dishes.wav', 'C:/data/tf_speech/train/audio/_background_noise_/exercise_bike.wav', 'C:/data/tf_speech/train/audio/_background_noise_/pink_noise.wav'}


In [21]:
from collections import Counter
count_train = Counter([s[0] for s in trainset_my])
for k in count_train.keys():
    print('{}:\t{},\t{}'.format(k, count_train[k], count_train[k] / len(trainset_my)))

0:	1776,	0.034197250356221356
1:	1780,	0.03427427119035699
2:	1733,	0.0333692763892633
3:	1738,	0.03346555243193284
4:	1733,	0.0333692763892633
5:	1729,	0.03329225555512766
6:	1769,	0.034062463896484
7:	1763,	0.03394693264528055
8:	1748,	0.03365810451727192
9:	1759,	0.03386991181114492
10:	4130,	0.07952401124504178
11:	30276,	0.5829706935726114


In [23]:
valfiles = [v[2] for v in valset]
[f.split('\\')[-2] + '_' + f.split('\\')[-1] for f in valfiles]

IndexError: list index out of range

['C:/data/tf_speech/train_modified/audio\\down\\00176480_nohash_0.wav',
 'C:/data/tf_speech/train_modified/audio\\down\\00f0204f_nohash_0.wav',
 'C:/data/tf_speech/train_modified/audio\\down\\014f9f65_nohash_0.wav',
 'C:/data/tf_speech/train_modified/audio\\down\\01d22d03_nohash_0.wav',
 'C:/data/tf_speech/train_modified/audio\\down\\037c445a_nohash_0.wav',
 'C:/data/tf_speech/train_modified/audio\\down\\03cf93b1_nohash_0.wav',
 'C:/data/tf_speech/train_modified/audio\\down\\03cf93b1_nohash_1.wav',
 'C:/data/tf_speech/train_modified/audio\\down\\0447d7c1_nohash_0.wav',
 'C:/data/tf_speech/train_modified/audio\\down\\0447d7c1_nohash_1.wav',
 'C:/data/tf_speech/train_modified/audio\\down\\0447d7c1_nohash_2.wav',
 'C:/data/tf_speech/train_modified/audio\\down\\05b2db80_nohash_0.wav',
 'C:/data/tf_speech/train_modified/audio\\down\\05b2db80_nohash_1.wav',
 'C:/data/tf_speech/train_modified/audio\\down\\06076b6b_nohash_0.wav',
 'C:/data/tf_speech/train_modified/audio\\down\\06076b6b_nohash_

In [26]:
fnames = set()
for s in valset:
    if s[0] == NAME2ID['silence']:
        fnames.add(s[2])
print([f.split('/') for f in fnames])

[[''], ['C:', 'data', 'tf_speech', 'train', 'audio', '_background_noise_', 'white_noise.wav'], ['C:', 'data', 'tf_speech', 'train', 'audio', '_background_noise_', 'running_tap.wav'], ['C:', 'data', 'tf_speech', 'train', 'audio', '_background_noise_', 'dude_miaowing.wav'], ['C:', 'data', 'tf_speech', 'train', 'audio', '_background_noise_', 'doing_the_dishes.wav'], ['C:', 'data', 'tf_speech', 'train', 'audio', '_background_noise_', 'exercise_bike.wav'], ['C:', 'data', 'tf_speech', 'train', 'audio', '_background_noise_', 'pink_noise.wav']]


In [4]:
from utilities import *
import pickle
import gzip

folds = pickle.load(gzip.open(TRAIN_MODIFIED_DIR + MY_KFOLD_FILENAME, 'rb'))

for fold in folds:
    trainset, valset = load_fold_my_noise(fold)
    train_uids = [s[1] for s in trainset]
    val_uids = [s[1] for s in valset]
    
    print(val_uids[:10])
    
    for i in val_uids:
        if i != '' and i in train_uids:
            print('!!!: ', i)

There are 47804 train and 16070 val samples
Noise files:
doing_the_dishes.wav
dude_miaowing.wav
exercise_bike.wav
pink_noise.wav
running_tap.wav
white_noise.wav
There are 51854 train and 17432 val samples after adding SILENCE
['00176480', '00f0204f', '00f0204f', '0135f3f2', '02746d24', '03c96658', '0447d7c1', '060cd039', '060cd039', '0685264e']
There are 48125 train and 15749 val samples
Noise files:
doing_the_dishes.wav
dude_miaowing.wav
exercise_bike.wav
pink_noise.wav
running_tap.wav
white_noise.wav
There are 52199 train and 17081 val samples after adding SILENCE
['0132a06d', '01648c51', '01648c51', '016e2c6d', '01bcfc0c', '035de8fe', '0362539c', '0362539c', '0397ecda', '05d18852']
There are 47525 train and 16349 val samples
Noise files:
doing_the_dishes.wav
dude_miaowing.wav
exercise_bike.wav
pink_noise.wav
running_tap.wav
white_noise.wav
There are 51551 train and 17729 val samples after adding SILENCE
['012c8314', '012c8314', '0137b3f4', '014f9f65', '01b4757a', '01b4757a', '022799

# Create KFold with my noise. Other files - from my previous KFold

In [90]:
folds = pickle.load(gzip.open(TRAIN_MODIFIED_DIR + MY_KFOLD_FILENAME, 'rb'))

print('Without _background_noise_ files')
for f in folds:
    print(len(f[0]), len(f[1]))
    
    count = 0
    for fname in f[0]:
        splits = fname.split('\\')
        label, uid = splits[-2], splits[-1].split('_')[0]
        if label == '_background_noise_':
            continue
        count += 1
    print('Train: ', count)
    
    count_val = 0
    for fname in f[1]:
        splits = fname.split('\\')
        label, uid = splits[-2], splits[-1].split('_')[0]
        if label == '_background_noise_':
            continue
        count += 1
        count_val += 1
    print('Val: ', count_val)
    print('Total: ', count)
    print('-----------------')
n_samples = count

Without _background_noise_ files
54171 18056
Train:  47804
Val:  16070
Total:  63874
-----------------
54556 17671
Train:  48125
Val:  15749
Total:  63874
-----------------
53922 18305
Train:  47525
Val:  16349
Total:  63874
-----------------
54599 17628
Train:  48168
Val:  15706
Total:  63874
-----------------


In [158]:
BG_MODIFIED_DIR = TRAIN_MODIFIED_DIR + 'audio\\_background_noise_\\'

def load_bg_noises():
    bg_files = os.listdir(BG_DIR)
    bg_files.remove('README.md')
    bg_files = [os.path.join(BG_MODIFIED_DIR, f) for f in bg_files]
    print(bg_files)
    bg_wavs = []
    for bg_file in bg_files:
        _, wav = wavfile.read(bg_file)
        bg_wavs.append(wav)
    return bg_wavs, bg_files

bg_wavs, bg_files = load_bg_noises()

['C:/data/tf_speech/train_modified/audio\\_background_noise_\\doing_the_dishes.wav', 'C:/data/tf_speech/train_modified/audio\\_background_noise_\\dude_miaowing.wav', 'C:/data/tf_speech/train_modified/audio\\_background_noise_\\exercise_bike.wav', 'C:/data/tf_speech/train_modified/audio\\_background_noise_\\pink_noise.wav', 'C:/data/tf_speech/train_modified/audio\\_background_noise_\\running_tap.wav', 'C:/data/tf_speech/train_modified/audio\\_background_noise_\\white_noise.wav']


C:\Users\MAX\Anaconda2\envs\tf\lib\site-packages\scipy\io\wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [159]:
for w in bg_wavs:
    print(w.dtype)

int16
int16
int16
int16
int16
int16


In [52]:
noise_type_percent = (SILENCE_PERCENT / (1 - SILENCE_PERCENT)) / (len(bg_wavs) + 1)
n_noise_type = int(noise_type_percent * n_samples)
print('Adding', n_noise_type, 'samples for each noise file')

Adding 902 samples for each noise file


In [160]:
new_bg_wavs = []
new_bg_files = []

for wav, f in zip(bg_wavs, bg_files):
    count = 0
    f_splits = f.split('.')
    for i in range(n_noise_type):
        i = np.random.randint(0, len(wav) - LENGTH)
        new_wav = wav[i : i + LENGTH]
        new_bg_wavs.append(new_wav)
        new_f = f_splits[-2] + '_{}.'.format(count) + f_splits[-1]
        new_bg_files.append(new_f)
        wavfile.write(new_f, LENGTH, new_wav)
        count += 1

In [93]:
new_bg_files = np.array(new_bg_files)

In [107]:
new_bg_files

array(['C:/data/tf_speech/train_modified/audio/_background_noise_/doing_the_dishes_0.wav',
       'C:/data/tf_speech/train_modified/audio/_background_noise_/doing_the_dishes_1.wav',
       'C:/data/tf_speech/train_modified/audio/_background_noise_/doing_the_dishes_2.wav',
       ...,
       'C:/data/tf_speech/train_modified/audio/_background_noise_/white_noise_899.wav',
       'C:/data/tf_speech/train_modified/audio/_background_noise_/white_noise_900.wav',
       'C:/data/tf_speech/train_modified/audio/_background_noise_/white_noise_901.wav'],
      dtype='<U82')

In [117]:
new_bg_files = glob.glob(os.path.join(BG_MODIFIED_DIR, '*wav'))
print(len(new_bg_files))
new_bg_files.remove('C:/data/tf_speech/train_modified/audio\\_background_noise_\\doing_the_dishes.wav')
new_bg_files.remove('C:/data/tf_speech/train_modified/audio\\_background_noise_\\dude_miaowing.wav')
new_bg_files.remove('C:/data/tf_speech/train_modified/audio\\_background_noise_\\exercise_bike.wav')
new_bg_files.remove('C:/data/tf_speech/train_modified/audio\\_background_noise_\\pink_noise.wav')
new_bg_files.remove('C:/data/tf_speech/train_modified/audio\\_background_noise_\\running_tap.wav')
new_bg_files.remove('C:/data/tf_speech/train_modified/audio\\_background_noise_\\white_noise.wav')
print(len(new_bg_files))
new_bg_files = np.array(new_bg_files)

5418
5412


In [118]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=17)
bg_folds = []
for train_index, test_index in kf.split(new_bg_files):
    bg_folds.append([new_bg_files[train_index], new_bg_files[test_index]])

In [119]:
for bg_fold in bg_folds:
    for f in bg_fold[0]:
        if f in bg_fold[1]:
            print('!!!: ', f)

In [120]:
old_folds = pickle.load(gzip.open(TRAIN_MODIFIED_DIR + MY_KFOLD_FILENAME, 'rb'))
new_folds = []

for i, f in enumerate(old_folds):
    print(len(f[0]), len(f[1]))
    new_f_train = []
    new_f_val = []
    
    for fname in f[0]:
        splits = fname.split('\\')
        label, uid = splits[-2], splits[-1].split('_')[0]
        if label != '_background_noise_':
            new_f_train.append(fname)

    for fname in f[1]:
        splits = fname.split('\\')
        label, uid = splits[-2], splits[-1].split('_')[0]
        if label != '_background_noise_':
            new_f_val.append(fname)

    for fname in bg_folds[i][0]:
        new_f_train.append(fname)
        
    for fname in bg_folds[i][1]:
        new_f_val.append(fname)
    
    new_folds.append([new_f_train, new_f_val])

54171 18056
54556 17671
53922 18305
54599 17628


In [121]:
for f in new_folds:
    print(len(f[0]), len(f[1]))

51863 17423
52184 17102
51584 17702
52227 17059


In [122]:
from utilities import *
import pickle
import gzip

with gzip.open(TRAIN_MODIFIED_DIR + 'kfold4_max_new_noise.pklz', 'wb') as f:
    pickle.dump(new_folds, f)

In [123]:
MY_KFOLD_NOISE_FILENAME = 'kfold4_max_new_noise.pklz'

old_folds = pickle.load(gzip.open(TRAIN_MODIFIED_DIR + MY_KFOLD_FILENAME, 'rb'))
new_folds = pickle.load(gzip.open(TRAIN_MODIFIED_DIR + MY_KFOLD_NOISE_FILENAME, 'rb'))

In [124]:
new_folds[0][0][:10]

['C:/data/tf_speech/train_modified/audio\\bed\\004ae714_nohash_0.wav',
 'C:/data/tf_speech/train_modified/audio\\bed\\004ae714_nohash_1.wav',
 'C:/data/tf_speech/train_modified/audio\\bed\\012c8314_nohash_0.wav',
 'C:/data/tf_speech/train_modified/audio\\bed\\012c8314_nohash_1.wav',
 'C:/data/tf_speech/train_modified/audio\\bed\\0132a06d_nohash_0.wav',
 'C:/data/tf_speech/train_modified/audio\\bed\\0137b3f4_nohash_0.wav',
 'C:/data/tf_speech/train_modified/audio\\bed\\014f9f65_nohash_0.wav',
 'C:/data/tf_speech/train_modified/audio\\bed\\01648c51_nohash_0.wav',
 'C:/data/tf_speech/train_modified/audio\\bed\\01648c51_nohash_1.wav',
 'C:/data/tf_speech/train_modified/audio\\bed\\016e2c6d_nohash_0.wav']

In [125]:
new_folds[0][0][-10:]

['C:/data/tf_speech/train_modified/audio\\_background_noise_\\white_noise_9.wav',
 'C:/data/tf_speech/train_modified/audio\\_background_noise_\\white_noise_901.wav',
 'C:/data/tf_speech/train_modified/audio\\_background_noise_\\white_noise_91.wav',
 'C:/data/tf_speech/train_modified/audio\\_background_noise_\\white_noise_92.wav',
 'C:/data/tf_speech/train_modified/audio\\_background_noise_\\white_noise_93.wav',
 'C:/data/tf_speech/train_modified/audio\\_background_noise_\\white_noise_94.wav',
 'C:/data/tf_speech/train_modified/audio\\_background_noise_\\white_noise_95.wav',
 'C:/data/tf_speech/train_modified/audio\\_background_noise_\\white_noise_96.wav',
 'C:/data/tf_speech/train_modified/audio\\_background_noise_\\white_noise_97.wav',
 'C:/data/tf_speech/train_modified/audio\\_background_noise_\\white_noise_98.wav']

In [155]:
for i, f in enumerate(old_folds):
    for fname in f[0]:
        splits = fname.split('\\')
        label, uid = splits[-2], splits[-1].split('_')[0]
        if label != '_background_noise_':
            assert fname in new_folds[i][0]
            assert fname not in new_folds[i][1]
            
    for fname in f[1]:
        splits = fname.split('\\')
        label, uid = splits[-2], splits[-1].split('_')[0]
        if label != '_background_noise_':
            assert fname not in new_folds[i][0]
            assert fname in new_folds[i][1]

In [127]:
len(new_folds[0][0])

51863

In [128]:
fold = new_folds[0]
train_files = [f.split('\\')[-2] + f.split('\\')[-1] for f in fold[0]]
val_files = [f.split('\\')[-2] + f.split('\\')[-1] for f in fold[1]]

In [129]:
val_files[-10:]

['_background_noise_white_noise_868.wav',
 '_background_noise_white_noise_872.wav',
 '_background_noise_white_noise_874.wav',
 '_background_noise_white_noise_880.wav',
 '_background_noise_white_noise_882.wav',
 '_background_noise_white_noise_883.wav',
 '_background_noise_white_noise_890.wav',
 '_background_noise_white_noise_90.wav',
 '_background_noise_white_noise_900.wav',
 '_background_noise_white_noise_99.wav']

In [130]:
new_folds = pickle.load(gzip.open(TRAIN_MODIFIED_DIR + MY_KFOLD_NOISE_FILENAME, 'rb'))

In [131]:
for i, f in enumerate(new_folds):
    val_labels = []
    for fname in f[1]:
        splits = fname.split('\\')
        label, uid = splits[-2], splits[-1].split('_')[0]
        val_labels.append(label)
    print(np.unique(val_labels, return_counts=True))

(array(['_background_noise_', 'bed', 'bird', 'cat', 'dog', 'down', 'eight',
       'five', 'four', 'go', 'happy', 'house', 'left', 'marvin', 'nine',
       'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six',
       'stop', 'three', 'tree', 'two', 'up', 'wow', 'yes', 'zero'],
      dtype='<U18'), array([1986,  431,  446,  464,  453,  591,  609,  601,  573,  574,  454,
        437,  582,  458,  568,  567,  558,  555,  606,  591,  586,  437,
        564,  599,  558,  419,  577,  590,  465,  574,  583], dtype=int64))
(array(['_background_noise_', 'bed', 'bird', 'cat', 'dog', 'down', 'eight',
       'five', 'four', 'go', 'happy', 'house', 'left', 'marvin', 'nine',
       'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six',
       'stop', 'three', 'tree', 'two', 'up', 'wow', 'yes', 'zero'],
      dtype='<U18'), array([1922,  442,  431,  415,  428,  571,  539,  562,  579,  597,  427,
        419,  561,  444,  579,  578,  578,  595,  568,  538,  582,  437,
        603,  586,  

In [141]:
MY_KFOLD_NOISE_FILENAME = 'kfold4_max_new_noise.pklz'

new_folds = pickle.load(gzip.open(TRAIN_MODIFIED_DIR + MY_KFOLD_NOISE_FILENAME, 'rb'))

for i, f in enumerate(new_folds):
    val_labels = []
    for fname in f[1]:
        splits = fname.split('\\')
        label, uid = splits[-2], splits[-1].split('_')[0]
        if label =='_background_noise_':
            label = 'silence'
        if label not in LABELS:
            label = 'unknown'
        
        val_labels.append(label)
    val, c = np.unique(val_labels, return_counts=True)
    
    print('Fold:', i, len(f[0]), len(f[1]))
    print(val)
    print(c)
    print(np.sum(c))

Fold: 0 51863 17423
['down' 'go' 'left' 'no' 'off' 'on' 'right' 'silence' 'stop' 'unknown'
 'up' 'yes']
[  591   574   582   567   558   555   591  1353   599 10289   590   574]
17423
Fold: 1 52184 17102
['down' 'go' 'left' 'no' 'off' 'on' 'right' 'silence' 'stop' 'unknown'
 'up' 'yes']
[  571   597   561   578   578   595   538  1353   586 10040   538   567]
17102
Fold: 2 51584 17702
['down' 'go' 'left' 'no' 'off' 'on' 'right' 'silence' 'stop' 'unknown'
 'up' 'yes']
[  608   616   616   627   634   590   618  1353   611 10225   585   619]
17702
Fold: 3 52227 17059
['down' 'go' 'left' 'no' 'off' 'on' 'right' 'silence' 'stop' 'unknown'
 'up' 'yes']
[  559   546   556   575   551   584   573  1353   551 10011   610   590]
17059


In [ ]:
trainset, valset = load_fold_my_noise(old_folds)

